In [1]:
import pandas as pd
import numpy as np
import scipy
import random

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
file_name = '../data/AdSmartABdata.csv'
ad_df = pd.read_csv(file_name)

## Exposed and control Data

### clean out usesrs who don't respond to the experiment

In [3]:
clean_ad_df = ad_df[(ad_df['yes'] == 1) ^ (ad_df['no'] == 1)]
clean_ad_df['converted'] = clean_ad_df.yes
clean_control = clean_ad_df.groupby('experiment').get_group('control')
clean_exposed = clean_ad_df.groupby('experiment').get_group('exposed')
# clean_ad_df

In [4]:
summary = clean_ad_df.pivot_table(values='converted', index='experiment', aggfunc=np.sum)
summary['total'] = clean_ad_df.pivot_table(values='converted', index='experiment', aggfunc=lambda x: len(x))
summary['rate'] = clean_ad_df.pivot_table(values='converted', index='experiment')
summary

,converted,total,rate
experiment,,,
control,264,586,0.450512
exposed,308,657,0.468798


In [5]:

control_converted = summary['converted'][0]
control_total = summary['total'][0]
control_rate = summary['rate'][0]

exposed_converted = summary['converted'][1]
exposed_total = summary['total'][1]
exposed_rate = summary['rate'][1]

rate_difference = exposed_rate-control_rate
rate_difference

0.018285619295484168

## 1.2 classic A/B analysis

In [6]:

class ClassicalTest:
    def __init__(self, exposed, control, sample_size):
        self.exposed = np.array(exposed.sample(sample_size))
        self.control = np.array(control.sample(sample_size))
        self.sample_size = sample_size
        
    def z_value(self):
        e_mean = np.mean(self.exposed)
        c_mean = np.mean(self.control)
        #print("exposed mean:", e_mean, "control mean:", c_mean)
        z = (e_mean - c_mean)/(c_mean*(1-c_mean)/self.sample_size)**0.5
        return z

In [7]:
classic_test = ClassicalTest(clean_exposed.converted, clean_control.converted, 550)

In [8]:
classic_test.z_value()
print("z-value:", classic_test.z_value())
print("p-value:", scipy.stats.norm.sf(classic_test.z_value()))

z-value: 1.1148600461718396
p-value: 0.13245520013028161
